In [1]:
import xml.etree.ElementTree as ET 

In [2]:
def index_to_text(text, start, end):
    #extracting words from the text given the start and end indices
    str_idx_map = list(enumerate(text))
    string = ""
    for i in range(len(str_idx_map)):
        #print(start, x[i])
        if start in str_idx_map[i]:
            string = string + str_idx_map[i][1]
            #print(str_idx_map[i][1], string)
            if start < end :
                start = start + 1
    return string 

In [3]:
text = {}
lu = {}
pos_tag = {}
frame_name = {}
frame_element = {}
frame_element_lu = {}

In [4]:
path = "../ted_annotation_data/"
import os 
annotation = {}
languages = []
count = 0

for file in os.listdir(path): 
    print("-------------------------------------------------------------------------------") 
    filename = path + file 
    tree = ET.parse(filename)
    root = tree.getroot()
    lang = (file.split("_")[2]).split(".")[0]
    if lang not in languages:
        languages.append(lang)
        
    #creating separate dumps of the features
    if lang not in text:
        text[lang] = []
    
    if lang not in lu:
        lu[lang] = []
    
    if lang not in pos_tag:
        pos_tag[lang] = []
        
    if lang not in frame_name:
        frame_name[lang] = []
        
    if lang not in frame_element:
        frame_element[lang] = []
        
    if lang not in frame_element_lu:
        frame_element_lu[lang] = []
    
    for child in root:
        if child.tag == "header":
            continue
        #key = sentence id + language
        key = child.attrib["ID"] + "_" + lang
        annotation[key] = {}
        for child1 in child:
            if child1.tag == 'text':
                annotation[key]["text"] = child1.text
                text[lang].append((child1.text, child.attrib["ID"] ))
            else:
                if "lu" not in annotation[key]:
                        annotation[key]["lu"] = [child1.attrib["luName"].split(".")[0] ]
                else:
                        annotation[key]["lu"].append(child1.attrib["luName"].split(".")[0]) 
                #lu dump - contains the lu, corresponding luID and sentence ID
                lu[lang].append((child1.attrib["luName"].split(".")[0], child1.attrib["luID"], child.attrib["ID"]))
                
                
                
                if "pos_tag" not in annotation[key]:
                    annotation[key]["pos_tag"] = [child1.attrib["luName"].split(".")[1]]
                else:
                    annotation[key]["pos_tag"].append(child1.attrib["luName"].split(".")[1])
                
                #pos_tag dump
                pos_tag[lang].append((child1.attrib["luName"].split(".")[1], child1.attrib["luID"], child.attrib["ID"]))
                
                
                
                if "frame_name" not in annotation[key]:
                    annotation[key]["frame_name"] = [child1.attrib["frameName"]]
                else:
                    annotation[key]["frame_name"].append(child1.attrib["frameName"])
                #frame dump
                frame_name[lang].append(child1.attrib["frameName"])
                
            for child2 in child1:
                if child2.attrib['name'] == 'target':
                    continue
                #print(child2.tag, child2.attrib)
                else:
                    if "frame_element" not in annotation[key]:
                        annotation[key]["frame_element"] = []
                        annotation[key]["frame_element_lu"] = []
                    fe = []
                    fe_lu = []
                    for child3 in child2:
                        fe.append(child3.attrib["name"].replace("\n", ""))
                        #frame element dump
                        frame_element[lang].append(child3.attrib["name"].replace("\n", ""))
                        
                        fe_lu.append(index_to_text(annotation[key]["text"].replace("\n", ""), int(child3.attrib["start"]), int(child3.attrib["end"])))
                        #frame element lu dump
                        frame_element_lu[lang].append((index_to_text(annotation[key]["text"].replace("\n", ""), int(child3.attrib["start"]), int(child3.attrib["end"])), child3.attrib['ID'], child.attrib["ID"]))
                    
                    annotation[key]["frame_element"].append(fe)
                    annotation[key]["frame_element_lu"].append(fe_lu)

-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------


### The "annotation" dictionary is arranged as such : 
* **Annotation** { s_id : 	FName : [ 1, 2, 3 ...]
			LU : [1, 2, 3 ... ]
			POS_FN : [1, 2, 3 …. ]
			FE : [ 1( a, b, ..), 2( a, b, ..), ... ]
			FE_LU : [ 1( a, b, ..), 2( a, b, ..), ... ]
			Text : 
	} 


In [5]:
import pickle as pkl

with open("../resources/annotated_data.pickle", "wb") as pkl_out:
    pkl.dump(annotation, pkl_out)
    pkl.dump(text, pkl_out)
    pkl.dump(lu, pkl_out)
    pkl.dump(pos_tag, pkl_out)
    pkl.dump(frame_name, pkl_out)
    pkl.dump(frame_element, pkl_out)
    pkl.dump(frame_element_lu, pkl_out)
    pkl.dump(languages, pkl_out)